In [7]:
import numpy as np

from sklearn.cross_validation import cross_val_score


In [97]:
import random
print random.randint(1,3)

2


In [2]:
f2 = open('/Users/ynh3/Documents/estR/rel_counts','rb')
rel_judged = []
rel_label = []
for row in f2:
    words = row.split()
    rel_judged.append(float(words[3]))
    
    if float(words[3]) <= 50.0:
        rel_label.append(1)
    elif float(words[3]) > 50 and float(words[3]) <= 150:
        rel_label.append(2)
    else:
        rel_label.append(3)
        
print rel_judged
print rel_label

[75.0, 284.0, 367.0, 106.0, 578.0, 619.0, 280.0, 108.0, 339.0, 45.0, 154.0, 167.0, 166.0, 171.0, 285.0, 151.0, 364.0, 148.0, 33.0, 18.0, 190.0, 61.0, 135.0, 275.0, 252.0, 168.0, 75.0, 74.0, 176.0, 211.0, 29.0, 360.0, 453.0, 316.0, 581.0, 139.0, 90.0, 86.0, 113.0, 166.0, 22.0, 44.0, 105.0, 174.0, 164.0, 582.0, 205.0, 363.0, 189.0, 211.0]
[2, 3, 3, 2, 3, 3, 3, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 2, 1, 1, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3]


In [116]:
corpus=[]
target = []

train_corpus=[]
train_label=[]
test_corpus=[]
test_label=[]

#target_counter = 0
temp=[]


#for traces of each system
for i in range(0,100):
    
    f = open('/Users/ynh3/Documents/estR/traces/multi35/trace-'+ str(i),'rb')
    #f_output = open('/Users/alinamazi/Documents/estR/estimates/ML/classification/test-'+ str(i), 'w')
    n=5
    
    next(f)
    count = 51
    
    for row in f:
        #initialization
        prev_marked=[]
        num_captured=[]
        marked_caught=[]
        new_marked=[]
        
        index = count - 51
        
        marked_sofar = 0
        for j in range(0,n):
            if j > 0: row = next(f)
            line = row.strip()   
            if j==0: system = line.partition(' ')[0]

            line = line.partition(' ')[2]
            total_rel_this_round = line.split(' ')[5]
            new_rel_this_round = line.split(' ')[6]
            rel_overlap_this = float(total_rel_this_round) - float(new_rel_this_round)
            
            prev_marked.append(marked_sofar)
            num_captured.append(total_rel_this_round)
            marked_caught.append(rel_overlap_this)
            new_marked.append(float(new_rel_this_round))
            
            marked_sofar += float(total_rel_this_round) - float(rel_overlap_this)
            
            features = float(total_rel_this_round)#, float(new_rel_this_round) #, marked_sofar
            temp=[]
            temp.append(features)
            corpus.append(temp)
            target.append(rel_label[index])
            '''if rel_judged[index] <= 50:
                target.append(1)
            elif rel_judged[index] > 50 and rel_judged[index] <= 150:
                target.append(2)
            else:
                target.append(3)'''
            
            if count % 2 == 0:
                test_corpus.append(temp)
                #test_label.append(rel_label[index])
                test_label.append(random.randint(1,3))
            else:
                train_corpus.append(temp)
                train_label.append(rel_label[index])
                #train_label.append(random.randint(1,3))
        #print prev_marked
        #print num_captured
        #print marked_caught
        #print new_marked
        ##AER_arr.append(AER(N,float(rel_judged[index]))) 
        #lines.append('{} {} {} {}\n'.format(system, count, N, rel_judged[index]))
        count = count +1
        if count > 100: 
             count = 51
'''print corpus[:20] 
print target[:20]

print train_corpus[:10]
print train_label[:10]

print test_corpus[:10]
print test_label[:10]'''

'print corpus[:20] \nprint target[:20]\n\nprint train_corpus[:10]\nprint train_label[:10]\n\nprint test_corpus[:10]\nprint test_label[:10]'

In [100]:
train_corpus[:10]

[[19.0], [12.0], [17.0], [22.0], [9.0], [24.0], [22.0], [2.0], [1.0], [29.0]]

In [117]:
from sklearn import tree
cl_tree = tree.DecisionTreeClassifier()
cl_tree = cl_tree.fit(train_corpus, train_label)
#returns mean accuracy
cl_tree.score(test_corpus, test_label)

0.33483333333333332

In [8]:
clf = tree.DecisionTreeClassifier(random_state=0)
scores = cross_val_score(clf, corpus, target, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.00)


In [9]:
sum(i == 1 for i in target)

54000

In [52]:
sum(i ==2 for i in target)

1170

In [10]:
sum(i == 3 for i in target)

2790

In [110]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
cl_kn = neigh.fit(train_corpus, train_label)
cl_kn.score(test_corpus, test_label)

0.33346666666666669

In [11]:
scores = cross_val_score(cl_kn, corpus, target, cv=2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.55 (+/- 0.07)


In [118]:
from sklearn.naive_bayes import GaussianNB
cl_nb = GaussianNB()
cl_nb.fit(train_corpus, train_label)
cl_nb.score(test_corpus, test_label)

0.33483333333333332

In [81]:
scores = cross_val_score(cl_nb, corpus, target, cv=2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.00)


In [119]:
from sklearn.ensemble import AdaBoostClassifier

cl_ada = AdaBoostClassifier(n_estimators=100)
cl_ada.fit(train_corpus, train_label)
cl_ada.score(test_corpus, test_label)
#scores = cross_val_score(cl_ada, corpus, target)
#scores.mean()

0.33483333333333332

In [ ]:
from sklearn.svm import SVC
cl_svm = SVC()
cl_svm.fit(train_corpus, train_label)
cl_svm.score(test_corpus, test_label)

In [42]:
scores = cross_val_score(cl_svm, corpus, target, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.01)


In [113]:
from sklearn.ensemble import RandomForestClassifier
cl_ranf = RandomForestClassifier()
cl_ranf.fit(train_corpus, train_label)
cl_ranf.score(test_corpus, test_label)

0.33593333333333331

In [25]:
scores = cross_val_score(cl_ranf, corpus, target, cv=2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.00)


In [121]:
from sklearn.naive_bayes import MultinomialNB
cl_mnnb = MultinomialNB()
cl_mnnb.fit(train_corpus, train_label)
cl_mnnb.score(test_corpus, test_label)

0.33483333333333332

In [120]:
from sklearn import linear_model
cl_logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
cl_logreg.fit(train_corpus, train_label)
cl_logreg.score(test_corpus, test_label)

0.33483333333333332